In [ ]:
# ==============================
# 1. INSTALL + IMPORT LIBRARIES
# ==============================
%pip install scikit-learn -q

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

plt.style.use("seaborn-v0_8")

print("Libraries loaded.")


# ==============================
# 2. LOAD DATA
# ==============================
sentiment = pd.read_csv("../data/fear_greed_index.csv")
trades = pd.read_csv("../data/historical_data.csv")

# Standardize column names
sentiment.columns = sentiment.columns.str.strip().str.lower()
trades.columns = trades.columns.str.strip().str.lower()

print("\nSentiment dataset preview:")
print(sentiment.head(3))

print("\nTrades dataset preview:")
print(trades.head(3))


# ==============================
# 3. DATE CLEANING
# ==============================

# ---- SENTIMENT DATE ----
if "timestamp ist" in sentiment.columns:
    sentiment["date"] = pd.to_datetime(sentiment["timestamp ist"], errors="coerce")
elif "timestamp" in sentiment.columns:
    sentiment["date"] = pd.to_datetime(sentiment["timestamp"], errors="coerce")
elif "date" in sentiment.columns:
    sentiment["date"] = pd.to_datetime(sentiment["date"], errors="coerce")
else:
    raise KeyError("Sentiment dataset missing date column")

# Classification column
if "classification" not in sentiment.columns:
    raise KeyError("Sentiment dataset must contain 'classification' column")

sentiment.rename(columns={"classification": "Classification"}, inplace=True)


# ---- TRADES DATE ----
if "date" in trades.columns:
    trades["date"] = pd.to_datetime(trades["date"], errors="coerce")
elif "timestamp" in trades.columns:
    trades["date"] = pd.to_datetime(trades["timestamp"], unit="s", errors="coerce")
else:
    raise KeyError("Trades dataset needs date or timestamp column")

# Remove rows with invalid/out-of-bounds dates
trades = trades[trades["date"].notna()]
trades = trades[(trades["date"].dt.year >= 2000) & (trades["date"].dt.year <= 2100)]


# ==============================
# 4. MERGE DATASETS (CORRECT WAY)
# ==============================
sentiment_clean = sentiment[["date", "Classification"]].drop_duplicates()

df = trades.merge(
    sentiment_clean,
    on="date",
    how="left",
    validate="many_to_one",
)

print("\nMerged dataset shape:", df.shape)
print(df[["date", "Classification"]].drop_duplicates().head())

print("\nMissing values:")
print(df.isnull().sum())

df = df.drop_duplicates()
df = df.dropna(subset=["Classification"])

print("\nColumns available:")
print(df.columns.tolist())


# ==============================
# 5. FEATURE ENGINEERING
# ==============================

# Fix PnL column naming automatically
for col in df.columns:
    if col.lower().replace(" ", "") == "closedpnl":
        df.rename(columns={col: "closedPnL"}, inplace=True)

# Standardize size column
for col in df.columns:
    if "size" in col.lower() and "tokens" in col.lower():
        df.rename(columns={col: "size"}, inplace=True)
        break

# Create leverage column if it doesn't exist
if "leverage" not in df.columns:
    df["leverage"] = 1.0  # Default leverage value

# Win flag
df["win"] = df["closedPnL"] > 0


# ==============================
# 6. DAILY TRADER METRICS
# ==============================
daily = (
    df.groupby(["account", "date", "Classification"])
    .agg(
        daily_PnL=("closedPnL", "sum"),
        num_trades=("closedPnL", "count"),
        total_size=("size", "sum"),
        avg_size=("size", "mean"),
        avg_leverage=("leverage", "mean"),
        long_trades=("side", lambda x: (x == "buy").sum()),
        short_trades=("side", lambda x: (x == "sell").sum()),
        win_days=("win", "mean"),
    )
    .reset_index()
)

daily["long_ratio"] = daily["long_trades"] / daily["short_trades"].replace(0, 1)

print("\nDaily metrics:")
print(daily.head())


# ==============================
# 7. ACCOUNT LEVEL STATS
# ==============================
account_stats = (
    daily.groupby("account")
    .agg(
        total_PnL=("daily_PnL", "sum"),
        avg_daily_PnL=("daily_PnL", "mean"),
        total_trades=("num_trades", "sum"),
        avg_daily_trades=("num_trades", "mean"),
        avg_leverage=("avg_leverage", "mean"),
        win_rate=("win_days", "mean"),
        n_days=("daily_PnL", "count"),
    )
    .reset_index()
)

# Segmentation
th_lvg = account_stats["avg_leverage"].median()
account_stats["leverage_category"] = np.where(
    account_stats["avg_leverage"] > th_lvg,
    "High leverage",
    "Low leverage",
)

th_freq = account_stats["avg_daily_trades"].median()
account_stats["trading_frequency"] = np.where(
    account_stats["avg_daily_trades"] > th_freq,
    "Frequent",
    "Infrequent",
)

account_stats["consistency"] = np.where(
    account_stats["win_rate"] > 0.6,
    "Consistent",
    "Inconsistent",
)

daily = daily.merge(
    account_stats[
        ["account", "leverage_category", "trading_frequency", "consistency"]
    ],
    on="account",
    how="left",
)


# ==============================
# 8. PERFORMANCE BY SENTIMENT
# ==============================
perf_by_sentiment = (
    daily.groupby("Classification")
    .agg(
        avg_daily_PnL=("daily_PnL", "mean"),
        median_daily_PnL=("daily_PnL", "median"),
        std_daily_PnL=("daily_PnL", "std"),
        win_rate=("daily_PnL", lambda x: (x > 0).mean()),
        avg_trades_per_day=("num_trades", "mean"),
        avg_leverage=("avg_leverage", "mean"),
    )
    .round(3)
)

print("\nPerformance by sentiment:")
print(perf_by_sentiment)


# ==============================
# 9. VISUALIZATIONS
# ==============================
sns.boxplot(data=daily, x="Classification", y="daily_PnL")
plt.title("Daily PnL by Fear vs Greed")
plt.show()

plt.figure(figsize=(10, 5))
sns.lineplot(
    data=daily,
    x="date",
    y="num_trades",
    hue="Classification",
    estimator="mean",
    errorbar=None,
)
plt.title("Avg trades per day by sentiment")
plt.xticks(rotation=45)
plt.show()

plt.figure(figsize=(8, 5))
sns.barplot(data=daily, x="Classification", y="avg_leverage")
plt.title("Avg leverage by sentiment")
plt.show()


# ==============================
# 10. MACHINE LEARNING MODEL
# ==============================
daily = daily.sort_values(["account", "date"])

daily["lag_PnL_1"] = daily.groupby("account")["daily_PnL"].shift(1)
daily["lag_trades_1"] = daily.groupby("account")["num_trades"].shift(1)

bins = [-np.inf, -1, 1, np.inf]
labels = ["Negative", "Neutral", "Positive"]

daily["next_PnL_bucket"] = pd.cut(
    daily.groupby("account")["daily_PnL"].shift(-1),
    bins=bins,
    labels=labels,
)

features = [
    "Classification",
    "lag_PnL_1",
    "lag_trades_1",
    "avg_leverage",
    "leverage_category",
    "consistency",
]

X = daily[features].dropna(subset=["lag_PnL_1", "next_PnL_bucket"]).copy()
y = daily.loc[X.index, "next_PnL_bucket"]

X = pd.get_dummies(
    X,
    columns=["Classification", "leverage_category", "consistency"],
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

model = RandomForestClassifier(
    n_estimators=100,
    min_samples_leaf=5,
    max_features="sqrt",
    random_state=42,
)

model.fit(X_train, y_train)

print("\nTrain Score:", model.score(X_train, y_train))
print("Test Score:", model.score(X_test, y_test))

print("\nClassification Report:")
print(classification_report(y_test, model.predict(X_test)))


[notice] A new release of pip is available: 25.0.1 -> 26.0.1
[notice] To update, run: C:\Users\Admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.
Libraries loaded.

Sentiment dataset preview:
    timestamp  value classification        date
0  1517463000     30           Fear  2018-02-01
1  1517549400     15   Extreme Fear  2018-02-02
2  1517635800     40           Fear  2018-02-03

Trades dataset preview:
                                      account  coin  execution price  \
0  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9769   
1  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9800   
2  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9855   

   size tokens  size usd side     timestamp ist  start position direction  \
0       986.87   7872.16  BUY  02-12-2024 22:50        0.000000       Buy   
1        16.00    127.68  BUY  02-12-2024 22:50      986.524596       Buy   
2       144.09   1150.63  BUY  02-12-2024 22:50     1002.518996       Buy   

   closed pnl                                   transaction hash     orde

KeyError: "Label(s) ['leverage', 'size'] do not exist"